In [ ]:
print("hello")

In [ ]:
!pip install oandapyV20

In [ ]:
from oandapyV20 import API    # Importing the API library
import oandapyV20.endpoints.orders as orders

account_id = "101-001-27691855-001"  # Replace with your account ID
access_token = "45d74503bbd2913e28c2a0098ba05554-58693ee27e51dbee19cc01366fb9481f"   # Replace with your API key

api = API(access_token=access_token)

data = {
  "order": {
    "timeInForce": "FOK",
    "instrument": "EUR_USD",
    "units": "100",             # Positive for buy, negative for sell
    "type": "MARKET",           # Type of the order
    "positionFill": "DEFAULT"
  }
}

order = orders.OrderCreate(accountID=account_id, data=data)
response = api.request(order)

print(response)


In [ ]:
from oandapyV20 import API
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.orders as orders

account_id = "YOUR_ACCOUNT_ID"  # Replace with your account ID
access_token = "YOUR_API_KEY"   # Replace with your API key

api = API(access_token=access_token)

# Fetch open positions
open_positions = positions.OpenPositions(accountID=account_id)
response = api.request(open_positions)

# For each position, create an order to close it
for position in response['positions']:
    instrument = position['instrument']
    units = position['long']['units'] if position['long']['units'] != '0' else position['short']['units']
    # If the position is long, we need to sell to close, and vice versa
    units_to_close = -int(units)

    data = {
        "order": {
            "timeInForce": "FOK",
            "instrument": instrument,
            "units": str(units_to_close),  # Negative to close the position
            "type": "MARKET",
            "positionFill": "REDUCE_ONLY"
        }
    }

    # Send the order
    order = orders.OrderCreate(accountID=account_id, data=data)
    api.request(order)

print("All positions closed.")


## buy , sell ,exit not implimented 


In [2]:
from flask import Flask, request, jsonify
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions

app = Flask(__name__)

account_id = "YOUR_ACCOUNT_ID"  # Replace with your account ID
access_token = "YOUR_API_KEY"   # Replace with your API key
api = API(access_token=access_token)

@app.route('/trade', methods=['POST'])
def trade():
    request_data = request.get_json()

    for item in request_data:
        symbol = item.get("symbol")
        units = item.get("units")
        side = item.get("side")
        exit_position = item.get("exit")

        if exit_position:
            # Implement logic to close all positions for the given symbol
            pass  # Replace with the logic to close positions
        else:
            # Place buy or sell order
            units = units if side == "buy" else f"-{units}"
            data = {
                "order": {
                    "timeInForce": "FOK",
                    "instrument": symbol,
                    "units": units,
                    "type": "MARKET",
                    "positionFill": "DEFAULT"
                }
            }
            order = orders.OrderCreate(accountID=account_id, data=data)
            api.request(order)

    return jsonify({"status": "success"})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/sly/PycharmProjects/onada/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/sly/PycharmProjects/onada/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1076, in launch_instance
    app.initialize(argv)
  File "/Users/sly/PycharmProjects/onada/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/sly/PycharmProjects/onada/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py"

SystemExit: 1

/Users/sly/PycharmProjects/onada/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!pip install flask 

## Exit all implimented 


In [ ]:
from flask import Flask, request, jsonify
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions

app = Flask(__name__)

account_id = "YOUR_ACCOUNT_ID"  # Replace with your account ID
access_token = "YOUR_API_KEY"   # Replace with your API key
api = API(access_token=access_token)

@app.route('/trade', methods=['POST'])
def trade():
    request_data = request.get_json()

    for item in request_data:
        symbol = item.get("symbol")
        units = item.get("units")
        side = item.get("side")
        exit_position = item.get("exit")

        if exit_position and exit_position == "true":
            # Fetch the open positions for the symbol
            open_positions = positions.PositionDetails(accountID=account_id, instrument=symbol)
            response = api.request(open_positions)

            # Check if there are any open positions
            if response['position']['long']['units'] != '0':
                # Close long position
                units_to_close = -int(response['position']['long']['units'])
                data = {
                    "order": {
                        "timeInForce": "FOK",
                        "instrument": symbol,
                        "units": str(units_to_close),
                        "type": "MARKET",
                        "positionFill": "REDUCE_ONLY"
                    }
                }
                order = orders.OrderCreate(accountID=account_id, data=data)
                api.request(order)

            if response['position']['short']['units'] != '0':
                # Close short position
                units_to_close = -int(response['position']['short']['units'])
                data = {
                    "order": {
                        "timeInForce": "FOK",
                        "instrument": symbol,
                        "units": str(units_to_close),
                        "type": "MARKET",
                        "positionFill": "REDUCE_ONLY"
                    }
                }
                order = orders.OrderCreate(accountID=account_id, data=data)
                api.request(order)

        else:
            # Place buy or sell order
            units = units if side == "buy" else f"-{units}"
            data = {
                "order": {
                    "timeInForce": "FOK",
                    "instrument": symbol,
                    "units": units,
                    "type": "MARKET",
                    "positionFill": "DEFAULT"
                }
            }
            order = orders.OrderCreate(accountID=account_id, data=data)
            api.request(order)

    return jsonify({"status": "success"})

if __name__ == '__main__':
    app.run(debug=True)
